<a href="https://colab.research.google.com/github/Antoniolucasf/ESTUDANDO-GIT/blob/master/script_crop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pybboxes==0.1.6


In [ ]:
!pip uninstall -y torch torchvision torchaudio


In [ ]:
!pip install albumentations==1.4.0


In [ ]:
from cv2.dnn import ClassificationModel
import cv2
import os
import pybboxes as pbx
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import patches
import albumentations as A
import json

COCO_ANNOT_PATH = "coloque o caminho do rótulo de suas imagens (json)"

with open(COCO_ANNOT_PATH, "r") as f:
    COCO_DATASET = json.load(f)

def busca_anotacao(nomeImg: str, dataset):
    image_id = None
    for img in dataset['images']:
        if img['file_name'] == nomeImg:
            image_id = img['id']
            break

    if image_id is None:
        return [], []

    bboxes = []
    classes = []

    for ann in dataset['annotations']:
        if ann['image_id'] == image_id:
            bboxes.append(ann['bbox'])
            classes.append(ann['category_id'])

    return bboxes, classes

def calculate_slice_bboxes(
    image_height: int,
    image_width: int,
    slice_height: int = 600,
    slice_width: int = 600,
    overlap_height_ratio: float = 0.2,
    overlap_width_ratio: float = 0.2,
):
    slice_bboxes = []

    y_step = int(slice_height * (1 - overlap_height_ratio))
    x_step = int(slice_width * (1 - overlap_width_ratio))

    y_positions = []
    x_positions = []

    # --- Calcular posições Y SEM duplicar ---
    y = 0
    while True:
        y_positions.append(y)
        if y + slice_height >= image_height:
            break
        y = min(y + y_step, image_height - slice_height)

    # --- Calcular posições X SEM duplicar ---
    x = 0
    while True:
        x_positions.append(x)
        if x + slice_width >= image_width:
            break
        x = min(x + x_step, image_width - slice_width)

    # --- Montar os BBoxes ---
    for y_min in y_positions:
        for x_min in x_positions:
            slice_bboxes.append([
                x_min,
                y_min,
                x_min + slice_width,
                y_min + slice_height
            ])

    return slice_bboxes

def voc_to_coco_bbox(x1, y1, x2, y2):
    w = x2 - x1
    h = y2 - y1
    return [float(x1), float(y1), float(w), float(h)]

def convert_coco_bbox(
    bbox: list,
    cls: int,
    img_size: tuple,
) -> tuple:
    """
    Converte bbox do formato COCO para Pascal VOC

    COCO: [x_min, y_min, width, height]
    VOC:  [xmin, ymin, xmax, ymax]
    """

    dh, dw = img_size
    x, y, w, h = bbox

    x1 = int(x)
    y1 = int(y)
    x2 = int(x + w)
    y2 = int(y + h)

    # Clamp para manter dentro da imagem
    if x1 < 0:
        x1 = 0
    if x2 > dw - 1:
        x2 = dw - 1
    if y1 < 0:
        y1 = 0
    if y2 > dh - 1:
        y2 = dh - 1

    return (cls, [x1, y1, x2, y2])

def carregar_anotacoes(labels, image_size, dataset):
    bboxs, clss = [], []

    bboxes, classes = busca_anotacao(labels, dataset)

    for bbox, cls in zip(bboxes, classes):
        cls, box_voc = convert_coco_bbox(
            bbox=bbox,
            cls=cls,
            img_size=image_size
        )
        bboxs.append(box_voc)
        clss.append(cls)

    return np.array(clss), np.array(bboxs)

def crop_dataset(
    path_img: str,
    size_crop: tuple,
    save_dir_img: str,
    save_json_path: str
) -> int:

    os.makedirs(save_dir_img, exist_ok=True)

    coco_output = {
        "images": [],
        "categories": [
            {
                "id": 1,
                "name": "bacilo",
                "supercategory": "bacillus"
            }
        ],
        "annotations": [],
    }

    CATEGORY_MAP = {7: 1}  # normalização

    img_list = sorted(os.listdir(path_img))
    split_height, split_width = size_crop

    image_id = 0
    annotation_id = 1

    for img_name in img_list:

        img_path = os.path.join(path_img, img_name)
        image = cv2.imread(img_path)

        if image is None:
            continue



        img_h, img_w, _ = image.shape

        clss, bbox_lb = carregar_anotacoes(
            img_name,
            image.shape[:2],
            COCO_DATASET
        )

        if len(bbox_lb) == 0:
            continue

        slices = calculate_slice_bboxes(
            img_h, img_w, split_height, split_width
        )

        for slice_box in slices:

            crop_transform = A.Compose(
                [A.Crop(*slice_box)],
                bbox_params=A.BboxParams(
                    format="pascal_voc",
                    label_fields=["labels"],
                    min_visibility=0.3,
                    min_area=10
                ),
            )

            try:
                cropped = crop_transform(
                    image=image,
                    bboxes=bbox_lb,
                    labels=clss
                )
            except Exception:
                continue

            if len(cropped["bboxes"]) == 0:
                continue

            # ----- salvar imagem -----
            base_name = os.path.splitext(img_name)[0]
            crop_name = f"split_0_id{image_id}_{base_name}.jpg"
            crop_path = os.path.join(save_dir_img, crop_name)

            cv2.imwrite(crop_path, cropped["image"])

            h_crop, w_crop, _ = cropped["image"].shape

            coco_output["images"].append({
                "file_name": crop_name,
                "height": h_crop,
                "width": w_crop,
                "id": image_id
            })

            # ----- salvar annotations -----
            for cls, (x1, y1, x2, y2) in zip(
                cropped["labels"], cropped["bboxes"]
            ):
                coco_bbox = voc_to_coco_bbox(x1, y1, x2, y2)

                cls_norm = CATEGORY_MAP.get(int(cls), 1)

                coco_output["annotations"].append({
                    "id": annotation_id,
                    "image_id": image_id,
                    "bbox": coco_bbox,
                    "area": coco_bbox[2] * coco_bbox[3],
                    "iscrowd": 0,
                    "category_id": cls_norm,
                    "segmentation": []
                })

                annotation_id += 1

            image_id += 1

    # ----- salvar JSON -----
    with open(save_json_path, "w") as f:
        json.dump(coco_output, f, indent=4)

    print(f"JSON COCO salvo em: {save_json_path}")
    return image_id



def main():

    path_img = "coloque o caminho da pasta das imagens"
    box_size = (600, 600)

    dir_img = "coloque o caminho no qual você deseja salvar as imagens cropadas"

    total = crop_dataset(
        path_img=path_img,
        size_crop=box_size,
        save_dir_img=dir_img,
        save_json_path="coloque o caminho no qual você deseja salvar o rótulo das imagens cropadas"
    )

    print(f"Total de cortes gerados: {total}")

main()
